### En este notebook finalmente probaré si un Topic extraction no supervisado funciona mejor que los anteriores modelos.

In [2]:
import spacy

In [5]:
nlp = spacy.load('es_core_news_md', disable = ['parser', 'ner'])
stop_words = nlp.Defaults.stop_words

def lemmatize_doc(text, allowed_postags=['NOUN', '¡ADJ', 'VERB', 'ADV', 'PROPN']):
    text_out = [t.lemma_.lower() for t in nlp(text)
                if t.po_ in allowed_postags
                and len(t.lemma_)>3
                and not t.is_stop]
    return text_out

def build_texts(fname):
    with open(fname) as f:
        for line in f:
            yield lemmatize_doc(line)

In [6]:
lee_data_file = 'sem_eval_train_es_topic.csv'

In [7]:
with open(lee_data_file) as f:
    for line in f:
        print(line)
        break

2018-Es-01643,"@aliciaenp Ajajjaa somos del clan twitteras perdidas pa eventos ""importantes"" "

